# TVKit Comprehensive Sample Notebook

**TVKit** is a Python library for TradingView's financial data APIs with real-time WebSocket streaming and comprehensive data export capabilities.

## Features Covered in This Notebook

- **OHLCV Data Fetching** - Historical and real-time financial data
- **Data Export System** - Multiple formats (Polars DataFrame, JSON, CSV)
- **Financial Analysis** - Technical indicators and data analysis
- **Real-time Streaming** - Live market data updates
- **Multi-symbol Operations** - Working with multiple financial instruments

## Prerequisites

```bash
# Install tvkit with all dependencies
pip install tvkit polars matplotlib seaborn
```

In [28]:
# Import required libraries
import asyncio
import nest_asyncio
from datetime import datetime, timezone
from pathlib import Path

# Enable nested event loops for Jupyter notebooks
nest_asyncio.apply()

# TVKit imports
from tvkit.api.chart.ohlcv import OHLCV
from tvkit.export import DataExporter, ExportFormat
from tvkit.api.utils import convert_timestamp_to_iso

# Optional: Data analysis and visualization
try:
    import polars as pl
    import matplotlib.pyplot as plt
    import seaborn as sns
    ANALYSIS_AVAILABLE = True
    print("✅ Analysis libraries loaded successfully")
except ImportError as e:
    ANALYSIS_AVAILABLE = False
    print(f"⚠️  Analysis libraries not available: {e}")

print("🚀 TVKit sample notebook initialized!")

✅ Analysis libraries loaded successfully
🚀 TVKit sample notebook initialized!


```markdown
## Logging Configuration

The following cell configures logging and warning settings to ensure clean, readable notebook output:

- **Suppresses debug and info logs** from libraries such as `httpx`, `websockets`, and others, so only warnings and errors are shown.
- **Disables most warnings** to avoid cluttering the notebook with non-critical messages.
- This setup is recommended for interactive analysis, as it keeps the output focused on results and important issues.

You can adjust the logging level or re-enable warnings if you need more detailed troubleshooting information.
```

In [29]:
# Configure logging to suppress debug messages
import logging
import warnings

# Set logging levels to reduce verbosity
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("websockets").setLevel(logging.WARNING)

# Optionally suppress warnings
warnings.filterwarnings("ignore")

print("🔇 Debug logging disabled - clean output mode enabled")

🔇 Debug logging disabled - clean output mode enabled


## Basic OHLCV Data Fetching

Let's start by fetching historical OHLCV (Open, High, Low, Close, Volume) data for Apple stock (AAPL) from NASDAQ.

In [ ]:
async def fetch_historical_ohlcv_data():
    """Fetch historical OHLCV data for Apple stock."""
    async with OHLCV() as ohlcv:
        # Fetch last 100 daily bars for Apple
        ohlcv_data = await ohlcv.get_historical_ohlcv(
            exchange_symbol="NASDAQ:AAPL",
            interval="1D",  # Daily intervals
            bars_count=100
        )

    # Display basic information
    print(f"📊 Fetched {len(ohlcv_data)} OHLCV bars")
    print(f"📅 Date range: {convert_timestamp_to_iso(ohlcv_data[0].timestamp)} to {convert_timestamp_to_iso(ohlcv_data[-1].timestamp)}")

    # Show first few bars
    print("\n🔍 First 3 bars:")
    for i, bar in enumerate(ohlcv_data[:3]):
        print(f"  Bar {i+1}: {convert_timestamp_to_iso(bar.timestamp)[:10]} - Close: ${bar.close:.2f}, Volume: {bar.volume:,.0f}")

    return ohlcv_data

# Run the function
apple_data = await fetch_historical_ohlcv_data()

📊 Fetched 100 OHLCV bars
📅 Date range: 2025-07-29T18:20:00+00:00 to 2025-07-29T19:59:00+00:00

🔍 First 3 bars:
  Bar 1: 2025-07-29 - Close: $212.14, Volume: 9,422
  Bar 2: 2025-07-29 - Close: $212.09, Volume: 9,200
  Bar 3: 2025-07-29 - Close: $211.97, Volume: 8,067


## Data Export to Different Formats

TVKit's `DataExporter` class provides seamless export to multiple formats including Polars DataFrames, JSON, and CSV files.

In [48]:
async def demonstrate_data_export():
    """Demonstrate different data export formats."""
    exporter = DataExporter()

    # 1. Export to Polars DataFrame
    print("📈 Exporting to Polars DataFrame ...")
    df = await exporter.to_polars(apple_data, add_analysis=False)

    print(f"DataFrame shape: {df.shape}")
    print(f"Columns: {df.columns}")
    print("\n📋 First 5 rows:")
    print(df.head())

    # 2. Export to JSON file
    print("\n💾 Exporting to JSON file...")
    json_path = await exporter.to_json(
        apple_data,
        "./tvkit_exports/apple_ohlcv_data.json",
        include_metadata=True,
        indent=2
    )
    print(f"JSON exported to: {json_path}")

    # 3. Export to CSV file
    print("\n📊 Exporting to CSV file...")
    csv_path = await exporter.to_csv(
        apple_data,
        "./tvkit_exports/apple_ohlcv_data.csv",
        include_metadata=True,
        timestamp_format="iso"
    )
    print(f"CSV exported to: {csv_path}")

    return df

# Print Out of DataFrame
df = await demonstrate_data_export()

📈 Exporting to Polars DataFrame ...
DataFrame shape: (100, 6)
Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume']

📋 First 5 rows:
shape: (5, 6)
┌─────────────────────┬─────────┬─────────┬────────┬─────────┬────────┐
│ timestamp           ┆ open    ┆ high    ┆ low    ┆ close   ┆ volume │
│ ---                 ┆ ---     ┆ ---     ┆ ---    ┆ ---     ┆ ---    │
│ str                 ┆ f64     ┆ f64     ┆ f64    ┆ f64     ┆ f64    │
╞═════════════════════╪═════════╪═════════╪════════╪═════════╪════════╡
│ 2025-07-30T01:20:00 ┆ 212.24  ┆ 212.25  ┆ 212.14 ┆ 212.14  ┆ 9422.0 │
│ 2025-07-30T01:21:00 ┆ 212.14  ┆ 212.15  ┆ 212.07 ┆ 212.09  ┆ 9200.0 │
│ 2025-07-30T01:22:00 ┆ 212.1   ┆ 212.105 ┆ 211.97 ┆ 211.975 ┆ 8067.0 │
│ 2025-07-30T01:23:00 ┆ 211.965 ┆ 211.99  ┆ 211.95 ┆ 211.965 ┆ 3875.0 │
│ 2025-07-30T01:24:00 ┆ 211.96  ┆ 212.01  ┆ 211.91 ┆ 211.92  ┆ 6742.0 │
└─────────────────────┴─────────┴─────────┴────────┴─────────┴────────┘

💾 Exporting to JSON file...
JSON exported to: tvk

## Multi-Symbol Data Comparison

Let's fetch data for multiple symbols and compare their performance.

In [49]:
async def compare_multiple_symbols():
    """Fetch and compare data for multiple symbols."""
    symbols = [
        "NASDAQ:AAPL",  # Apple
        "NASDAQ:GOOGL", # Google
        "NASDAQ:MSFT",  # Microsoft
        "NYSE:TSLA"     # Tesla
    ]

    symbol_data = {}

    print("🔄 Fetching data for multiple symbols...")

    async with OHLCV() as ohlcv:
        for symbol in symbols:
            try:
                print(f"  📥 Fetching {symbol}...")
                data = await ohlcv.get_historical_ohlcv(
                    exchange_symbol=symbol,
                    interval="1D",
                    bars_count=30  # Last 30 days
                )
                symbol_data[symbol] = data
                print(f"    ✅ Got {len(data)} bars")
            except Exception as e:
                print(f"    ❌ Failed to fetch {symbol}: {e}")

    # Calculate performance metrics
    print("\n📊 Performance Summary (30-day period):")
    print("-" * 60)

    for symbol, data in symbol_data.items():
        if len(data) >= 2:
            first_close = data[0].close
            last_close = data[-1].close
            change_pct = ((last_close - first_close) / first_close) * 100

            avg_volume = sum(bar.volume for bar in data) / len(data)
            max_high = max(bar.high for bar in data)
            min_low = min(bar.low for bar in data)

            print(f"{symbol:12} | Change: {change_pct:+6.2f}% | "
                  f"Range: ${min_low:.2f}-${max_high:.2f} | "
                  f"Avg Vol: {avg_volume:,.0f}")

    return symbol_data

# Run multi-symbol comparison
multi_symbol_data = await compare_multiple_symbols()

🔄 Fetching data for multiple symbols...
  📥 Fetching NASDAQ:AAPL...
    ✅ Got 30 bars
  📥 Fetching NASDAQ:GOOGL...
    ✅ Got 30 bars
  📥 Fetching NASDAQ:MSFT...
    ✅ Got 30 bars
  📥 Fetching NYSE:TSLA...
    ❌ Failed to fetch NYSE:TSLA: Invalid exchange:symbol 'NYSE:TSLA' after 3 attempts

📊 Performance Summary (30-day period):
------------------------------------------------------------
NASDAQ:AAPL  | Change:  +6.48% | Range: $195.07-$216.23 | Avg Vol: 51,437,151
NASDAQ:GOOGL | Change: +10.74% | Range: $162.00-$197.95 | Avg Vol: 42,335,322
NASDAQ:MSFT  | Change:  +6.98% | Range: $472.51-$518.29 | Avg Vol: 18,472,857


## Cryptocurrency and Forex Data

TVKit supports various asset classes including cryptocurrencies and forex pairs.

In [33]:
async def fetch_crypto_and_forex_data():
    """Demonstrate fetching cryptocurrency and forex data."""

    # Different asset classes
    symbols = {
        "Cryptocurrency": [
            "BINANCE:BTCUSDT",  # Bitcoin
            "BINANCE:ETHUSDT",  # Ethereum
            "BINANCE:ADAUSDT"   # Cardano
        ],
        "Forex": [
            "FX_IDC:EURUSD",    # EUR/USD
            "FX_IDC:GBPUSD",    # GBP/USD
            "FX_IDC:USDJPY"     # USD/JPY
        ]
    }

    all_data = {}

    async with OHLCV() as ohlcv:
        for category, symbol_list in symbols.items():
            print(f"\n📊 Fetching {category} Data:")
            print("-" * 40)

            category_data = {}

            for symbol in symbol_list:
                try:
                    print(f"  📥 {symbol}...")
                    data = await ohlcv.get_historical_ohlcv(
                        exchange_symbol=symbol,
                        interval="240",  # 4-hour intervals
                        bars_count=50
                    )
                    category_data[symbol] = data

                    # Show latest price
                    latest = data[-1]
                    print(f"    ✅ Latest: ${latest.close:.6f} (Vol: {latest.volume:,.0f})")

                except Exception as e:
                    print(f"    ❌ Failed: {e}")

            all_data[category] = category_data

    # Calculate volatility for each asset
    print(f"\n📈 Volatility Analysis (4-hour intervals, last 50 bars):")
    print("-" * 60)

    for category, category_data in all_data.items():
        print(f"\n{category}:")
        for symbol, data in category_data.items():
            if len(data) > 1:
                # Calculate price volatility (standard deviation of returns)
                returns = []
                for i in range(1, len(data)):
                    ret = (data[i].close - data[i-1].close) / data[i-1].close
                    returns.append(ret)

                if returns:
                    volatility = (sum((r - sum(returns)/len(returns))**2 for r in returns) / len(returns))**0.5
                    volatility_pct = volatility * 100

                    print(f"  {symbol:20} | Volatility: {volatility_pct:.3f}% | Latest: ${data[-1].close:.6f}")

    return all_data

# Fetch crypto and forex data
crypto_forex_data = await fetch_crypto_and_forex_data()


📊 Fetching Cryptocurrency Data:
----------------------------------------
  📥 BINANCE:BTCUSDT...
    ✅ Latest: $117878.000000 (Vol: 434)
  📥 BINANCE:ETHUSDT...
    ✅ Latest: $3799.030000 (Vol: 15,651)
  📥 BINANCE:ADAUSDT...
    ✅ Latest: $0.777800 (Vol: 3,439,413)

📊 Fetching Forex Data:
----------------------------------------
  📥 FX_IDC:EURUSD...
    ✅ Latest: $1.155370 (Vol: 76,529)
  📥 FX_IDC:GBPUSD...
    ✅ Latest: $1.335000 (Vol: 81,428)
  📥 FX_IDC:USDJPY...
    ✅ Latest: $147.924000 (Vol: 132,855)

📈 Volatility Analysis (4-hour intervals, last 50 bars):
------------------------------------------------------------

Cryptocurrency:
  BINANCE:BTCUSDT      | Volatility: 0.568% | Latest: $117878.000000
  BINANCE:ETHUSDT      | Volatility: 1.202% | Latest: $3799.030000
  BINANCE:ADAUSDT      | Volatility: 1.541% | Latest: $0.777800

Forex:
  FX_IDC:EURUSD        | Volatility: 0.173% | Latest: $1.155370
  FX_IDC:GBPUSD        | Volatility: 0.144% | Latest: $1.335000
  FX_IDC:USDJPY    

## Real-time Data Streaming (Limited Demo)

⚠️ **Note**: Real-time streaming is demonstrated with a limited time window to prevent infinite loops in the notebook.

In [37]:
async def limited_realtime_demo():
    """Demonstrate real-time streaming with a time limit."""

    print("🚀 Starting limited real-time data stream (30 seconds)...")
    print("Symbol: BINANCE:BTCUSDT (Bitcoin)")
    print("-" * 50)

    start_time = asyncio.get_event_loop().time()
    timeout_seconds = 30  # Limit to 30 seconds
    bar_count = 0

    try:
        async with OHLCV() as ohlcv:
            async for bar in ohlcv.get_ohlcv("BINANCE:BTCUSDT", interval="1"):
                # Check timeout
                if asyncio.get_event_loop().time() - start_time > timeout_seconds:
                    print(f"\n⏰ Demo timeout reached ({timeout_seconds}s)")
                    break

                bar_count += 1
                timestamp_str = convert_timestamp_to_iso(bar.timestamp)

                print(f"📊 Bar {bar_count}: {timestamp_str} | "
                      f"Close: ${bar.close:,.2f} | "
                      f"Volume: {bar.volume:,.0f}")

                # Also limit by number of bars
                if bar_count >= 10:
                    print(f"\n📈 Received {bar_count} bars, stopping demo")
                    break

    except Exception as e:
        print(f"❌ Streaming error: {e}")

    print(f"\n✅ Real-time demo completed. Received {bar_count} bars.")

# Run limited real-time demo
await limited_realtime_demo()

🚀 Starting limited real-time data stream (30 seconds)...
Symbol: BINANCE:BTCUSDT (Bitcoin)
--------------------------------------------------
📊 Bar 1: 2025-07-30T05:13:00+00:00 | Close: $117,721.79 | Volume: 9
📊 Bar 2: 2025-07-30T05:14:00+00:00 | Close: $117,711.48 | Volume: 6
📊 Bar 3: 2025-07-30T05:15:00+00:00 | Close: $117,728.84 | Volume: 3
📊 Bar 4: 2025-07-30T05:16:00+00:00 | Close: $117,767.44 | Volume: 81
📊 Bar 5: 2025-07-30T05:17:00+00:00 | Close: $117,838.02 | Volume: 9
📊 Bar 6: 2025-07-30T05:18:00+00:00 | Close: $117,831.12 | Volume: 2
📊 Bar 7: 2025-07-30T05:19:00+00:00 | Close: $117,856.06 | Volume: 3
📊 Bar 8: 2025-07-30T05:20:00+00:00 | Close: $117,880.07 | Volume: 3
📊 Bar 9: 2025-07-30T05:21:00+00:00 | Close: $117,845.77 | Volume: 2
📊 Bar 10: 2025-07-30T05:22:00+00:00 | Close: $117,850.01 | Volume: 1

📈 Received 10 bars, stopping demo

✅ Real-time demo completed. Received 10 bars.


## Error Handling and Best Practices

Demonstration of proper error handling and best practices when working with TVKit.

In [50]:
async def demonstrate_error_handling():
    """Show proper error handling techniques with TVKit."""

    print("🛡️  Error Handling and Best Practices")
    print("=" * 45)

    # 1. Handle invalid symbols gracefully
    print("\n1️⃣  Invalid Symbol Handling:")
    invalid_symbols = ["INVALID:SYMBOL", "BADEXCHANGE:BADSTOCK"]

    async with OHLCV() as ohlcv:
        for symbol in invalid_symbols:
            try:
                print(f"  📥 Attempting to fetch {symbol}...")
                data = await ohlcv.get_historical_ohlcv(
                    exchange_symbol=symbol,
                    interval="1D",
                    bars_count=10
                )
                print(f"    ✅ Success: Got {len(data)} bars")
            except Exception as e:
                print(f"    ❌ Expected error: {type(e).__name__}: {e}")

    # 2. Handle network timeouts and connection issues
    print(f"\n2️⃣  Connection Resilience:")

    try:
        async with OHLCV() as ohlcv:
            # This should work normally
            data = await ohlcv.get_historical_ohlcv(
                exchange_symbol="NASDAQ:AAPL",
                interval="1D",
                bars_count=5
            )
            print(f"    ✅ Successfully fetched {len(data)} bars")
    except Exception as e:
        print(f"    ❌ Connection error: {e}")

    # 3. Export error handling
    print(f"\n3️⃣  Export Error Handling:")

    try:
        exporter = DataExporter()

        # Try to export to an invalid path
        result = await exporter.export_ohlcv_data(
            apple_data[:5],  # Use small subset
            ExportFormat.JSON,
            file_path="/invalid/path/cannot_write_here.json"
        )

        if result.success:
            print(f"    ✅ Export successful")
        else:
            print(f"    ❌ Export failed: {result.error_message}")

    except Exception as e:
        print(f"    ❌ Export exception: {type(e).__name__}: {e}")

    # 4. Best practices summary
    print(f"\n💡 Best Practices Summary:")
    print("   • Always use async context managers (async with)")
    print("   • Handle symbol validation errors gracefully")
    print("   • Set appropriate timeouts for real-time streams")
    print("   • Check export results for success status")
    print("   • Use try-except blocks for robust error handling")
    print("   • Validate data before processing")

# Demonstrate error handling
await demonstrate_error_handling()

🛡️  Error Handling and Best Practices

1️⃣  Invalid Symbol Handling:
  📥 Attempting to fetch INVALID:SYMBOL...
    ❌ Expected error: ValueError: Invalid exchange:symbol 'INVALID:SYMBOL' after 3 attempts
  📥 Attempting to fetch BADEXCHANGE:BADSTOCK...
    ❌ Expected error: ValueError: Invalid exchange:symbol 'BADEXCHANGE:BADSTOCK' after 3 attempts

2️⃣  Connection Resilience:


2025-07-30 12:34:00,890 - ERROR - Failed to export OHLCV data to JSON: [Errno 30] Read-only file system: '/invalid'


    ✅ Successfully fetched 5 bars

3️⃣  Export Error Handling:
    ❌ Export failed: [Errno 30] Read-only file system: '/invalid'

💡 Best Practices Summary:
   • Always use async context managers (async with)
   • Handle symbol validation errors gracefully
   • Set appropriate timeouts for real-time streams
   • Check export results for success status
   • Use try-except blocks for robust error handling
   • Validate data before processing


## Summary

This notebook has demonstrated the comprehensive capabilities of TVKit:

### ✅ Completed Examples

- **Basic OHLCV Data Fetching** - Retrieved historical market data for Apple stock
- **Multi-format Data Export** - Exported to Polars DataFrame, JSON, and CSV formats
- **Multi-symbol Operations** - Compared performance across multiple stocks
- **Cryptocurrency & Forex** - Demonstrated support for various asset classes
- **Real-time Streaming** - Limited demo of live data streaming
- **Error Handling** - Best practices for robust applications

### 🔧 Key Features Highlighted

- **Async Architecture** - All operations use modern async/await patterns
- **Type Safety** - Comprehensive Pydantic models for data validation
- **Multiple Asset Classes** - Stocks, crypto, forex, and more
- **Flexible Export System** - Support for Polars, JSON, CSV with custom options
- **Real-time Capabilities** - WebSocket streaming for live market data
- **Error Resilience** - Robust error handling and validation

### 📚 Next Steps

- Explore the full [TVKit documentation](https://github.com/your-repo/tvkit)
- Check out additional examples in the `examples/` directory
- Review the API reference for advanced features
- Consider integrating TVKit into your financial analysis workflows

**Happy Trading! 📈**